## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = 2020
month = 11
program = "default"
study_type = "program"
mode_of_interest = "e-bike"
include_test_users = False
dynamic_labels = { }
use_imperial = True

In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("The plots in this notebook are only relevant to programs")

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

# get metric vs imperial vars
label_units, short_label, label_units_lower, distance_col, weight_unit = scaffolding.get_units(use_imperial)

### Color Dictionary

In [ ]:
colors_mode, colors_purpose, colors_sensed = scaffolding.mapping_color_labels(dynamic_labels, dic_re, dic_pur)

## Collect Data From Database

In [ ]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dynamic_labels,
                                                                            dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

In [ ]:
if 'mode_confirm' in expanded_ct.columns:
    mode_of_interest_df = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'")
    debug_df.loc[f"{mode_of_interest}_trips"] = len(mode_of_interest_df)
    debug_df.loc[f"{mode_of_interest}_trips_with_replaced_mode"] = scaffolding.trip_label_count("Replaced_mode", mode_of_interest_df)

## Metrics for Specific Mode

In [ ]:
data_eb = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'") if "mode_confirm" in expanded_ct.columns else expanded_ct

In [ ]:
quality_text = scaffolding.get_quality_text(expanded_ct, data_eb, mode_of_interest, include_test_users)

### Trips by purpose for specified mode

In [ ]:
plot_title_no_quality = f"Number of trips for each purpose for {mode_of_interest}"
file_name= f"ntrips_{mode_of_interest}_purpose{file_suffix}"

try:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,2*1), sharex=True)
    text_results = ["Unmodified Alt Text", "Unmodified HTML"]
    plot_and_text_stacked_bar_chart(data_eb, lambda df: df.groupby("Trip_purpose").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False),
                                    f"Labeled `{mode_of_interest}` by user", ax, text_results, colors_purpose, debug_df)
    plot_title = plot_title_no_quality + "\n" + f"For {mode_of_interest}: " + quality_text
    set_title_and_save(fig, text_results, plot_title, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, debug_df, file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    
    alt_html = store_alt_html_missing(debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Total Trip Length for each mode replaced by the specified mode

In [ ]:
plot_title_no_quality = "Total trip length (" + label_units_lower + ") covered by replaced mode"
file_name = f"total_trip_length_{mode_of_interest}_replaced_mode{file_suffix}"

try:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,2*1), sharex=True)
    text_results = ["Unmodified Alt Text", "Unmodified HTML"]
    plot_and_text_stacked_bar_chart(data_eb, lambda df: df.groupby("Replaced_mode").agg({distance_col: 'sum'}).sort_values(by=distance_col, ascending=False), 
                                    "Labeled by user\n (Trip distance)", ax, text_results, colors_mode, debug_df)
    plot_title = plot_title_no_quality + "\n" + f"For {mode_of_interest}: " + quality_text
    set_title_and_save(fig, text_results, plot_title, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, debug_df, file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    
    alt_html = store_alt_html_missing(debug_df, file_name, plot_title_no_quality)
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Number of Trips for each mode replaced by specified mode

In [ ]:
plot_title_no_quality= "Number of trips for replaced mode"
file_name = f'ntrips_{mode_of_interest}_total{file_suffix}'

try:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,2*1), sharex=True)
    text_results = ["Unmodified Alt Text", "Unmodified HTML"]
    plot_and_text_stacked_bar_chart(data_eb, lambda df: df.groupby("Replaced_mode").agg({distance_col: 'count'}).sort_values(by=distance_col, ascending=False), 
                                    f"Labeled `{mode_of_interest}` by user", ax, text_results, colors_mode, debug_df)
    plot_title = plot_title_no_quality + "\n" + f"For {mode_of_interest}: " + quality_text
    set_title_and_save(fig, text_results, plot_title, file_name)
except (AttributeError, KeyError, pd.errors.UndefinedVariableError) as e:
    plt.clf()
    generate_missing_plot(plot_title_no_quality, debug_df, file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)    
    alt_html = store_alt_html_missing(debug_df, file_name, plot_title_no_quality)    
except Exception as e:
    fig, ax = plt.subplots()
    plot_and_text_error(e, ax, file_name)

### Average miles per trip for specified mode

In [ ]:
plot_title_no_quality="Average " + label_units + " for each replaced mode with > 3 entries\n'Other' represents trips with a non-standard or missing replacement"
file_name ='average_miles_replaced_mode%s' % file_suffix

try:
    dg=data_eb.groupby('Replaced_mode').agg({distance_col: ['sum', 'count' , 'mean']},)
    dg.columns = ['Total ('+label_units_lower+')', 'Count' ,'Average ('+label_units_lower+')']
    dg = dg.reset_index()
    dg = dg.sort_values(by=['Total ('+label_units_lower+')'], ascending=False)
    data = dg.drop((dg.query("Count < 3").index)).sort_values(by=['Average ('+label_units_lower+')'], ascending=False) 
        
    x='Replaced_mode'
    y='Average ('+label_units_lower+')'
    y2 = "Count"

    plot_title= plot_title_no_quality+"\n"+quality_text
    barplot_mode(data,x,y,plot_title, expanded_ct['Replaced_mode'].dropna().unique().tolist(), file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Average ('+label_units_lower+')'].values, data.Replaced_mode), file_name, plot_title)

except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Number of trips by day for specified mode

In [ ]:
plot_title_no_quality=f"Number of {mode_of_interest} trips by day"
file_name =f'ntrips_{mode_of_interest}_per_day%s' % file_suffix


try:
    fq_days = data_eb.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
    fq_days = fq_days.reset_index()
    fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']

    data = fq_days
    x = 'Day of the Month'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Day of the Month'].values), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Number of trips by day of week¶

In [ ]:
plot_title_no_quality=f"Number of {mode_of_interest} trips by weekday"
file_name =f'ntrips_{mode_of_interest}_per_weekday%s' % file_suffix

try:
    fq_weekdays = data_eb.groupby(['start_local_dt_weekday']).agg({'start_local_dt_weekday': ['sum', 'count']})
    fq_weekdays = fq_weekdays.reset_index()
    fq_weekdays.columns = ['Weekday', 'Total', 'Number of Trips']
    weekday_labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    fq_weekdays["Weekday"] = fq_weekdays.Weekday.apply(lambda x: weekday_labels[x])

    data = fq_weekdays
    x = 'Weekday'
    y = 'Number of Trips'

    plot_title= plot_title_no_quality+"\n"+quality_text
    barplot_day(data,x,y,plot_title,file_name)
    alt_text = store_alt_text_bar(pd.DataFrame(data['Number of Trips'].values, data['Weekday'].values), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)